# Modélisation

### 0 - importation des modules 

In [230]:
# Module pandas pour manipuler le dataframe ensuite constitué :
import pandas as pd

# Module numpy pour manipuler essentiellement des arrays :
import numpy as np
import matplotlib.pyplot as plt

# Module scikitlearn pour la modélisation : 
import sklearn

On importe notre propre module ***preprocessing_data*** qui regroupe les fonctions qui ont servi pour les étapes de preprocessing :

In [231]:
import sys
sys.path.append('./modules')
from modelisation import preprocessing_data

In [232]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1 - Sélection du modèle

Avant de se lancer dans les étapes de machine learning, il est important de bien comprendre ce que l'on souhaite prédire, et avec quelles variables, quelles sont les types de toutes ces variables, afin de comprendre quelle sera le modèle le plus adapté à notre jeu de données.

On peut commencer par importer à nouveau notre jeu de données :

In [233]:
df = pd.read_csv("data/data_cleaned.csv", sep = "\t")

In [234]:
len(df)

188255

In [235]:
df.head(3)

,tconst,title,year,runtime,genres,rate,votes,director,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."


Nous souhaitons prédire la note (spectateurs) d'un film : ***rate***. Il s'agit d'une variable quantitative, continue sur le segment [0,10].
Pour prédire cette note, nous allons utiliser différentes variables : 
- ***year*** : l'année de sortie du film (quantitative discrète)
- ***runtime*** : la durée du film (quantitative discrète)
- ***votes*** : le nombre de votes recensés pour former la note du film
- ***genres*** : le genre du film (qualitative)
- ***director*** : le réalisateur du film (qualitative)
- ***casting*** : les acteurs du film (liste de qualitatives)

Critère 1 : Quantité de données
Critère 2 : structure des données (ici données structurées)
Critère 3 : Variables qualitatives / quantitatives

Une regression linéaire semble tout à fait adaptée pour démarrer.

### 2 - Preprocessing

Avant de mettre en oeuvre une modélisation via un modèle de regression linéaire, certaines variables doivent être transformées.

Plus précisément, nous avons besoin de transformer les variables qualitatives **genres**, **casting**, et **director** en variables quantitatives.

Pour la variable **genres**, comme vu précédemment il y a seulement 28 genres présents dans la base de données, donc 28 modalités prises par la variable **genres**. Un endodage du type "***One Hot Encoding***" fera tout à fait l'affaire.

(expliquer le principe du One Hot Encoding)

#### 2.1 - Transformation de la variable ***genres***

(voir la video du mec pour voir comment bien faire)

#### 2.2 - Transformation des variables ***director*** et ***casting***

Nous voyons que la situation pour ces deux variables n'est pas la même que la pour la variable ***genres***. En effet un encodage du type "***One Hot Encoding***" risque d'être totalement inapproprié à la structure de ces deux variables. Plus préciséments, contrairement aux genres, les nombres de réalisateurs et acteurs sont très conséquents.

Par exemple pour les directeurs, combien y en a t il?

In [236]:
df["director"].nunique()

68373

Il y a 68 373 réalisateurs différents dans notre base de données. Le One Hot encoding va ainsi générer une sparse matrix de taille 68373x68373 ! Ce n'est pas adapté.

Nous avons ainsi trouvé une autre alternative : à partir d'une formule mathématique, nous allons affecter un score aux réalisateurs, basé sur les notes qu'on recues les films qu'ils ont produits et le nombre de films qu'ils ont fait.

La formule est la suivante : 

$$\frac{1}{rank}rate^2$$

Avec :

- ***rank*** : le classement du film dans la base de données (selon sa note)
- ***rate*** : la note du film par les telespectateurs (la variable **rate** de la base de données)

Ainsi pour un réalisateur donné, on obtient son score en faisant la somme de tous les scores que ses films lui ont fait obtenir.

Pour appliquer cette formule il faut bien evidemment calculer la variable rank pour chaque film. 

In [237]:
df = preprocessing_data.add_ranks_films(df)

On vérifie que le rank est bien apparu dans la base, c'est le cas, par exemple on voit que le film "Rebecca" est classée 98 476ème (c'est à dire 98 476ème meilleure note) :

In [238]:
df.head(5)

,tconst,title,year,runtime,genres,rate,votes,director,casting,index,rank
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",0,33273.0
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",1,6429.0
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",2,98476.0
3,tt10682266,Hubie Halloween,2020,102,"Comedy, Fantasy, Mystery",5.2,27526,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',...",3,140355.0
4,tt2222042,Love and Monsters,2020,109,"Action, Adventure, Comedy",7.1,10780,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ...",4,28509.0


Maintenant qu'on a le rank, on peut calculer le score de chaque réalisateur. C'est ce qui a été implémenté dans la fonction ***add_score_realisateurs*** :

In [239]:
df = preprocessing_data.add_scores_realisateurs(df)

On peut vérifier que la fonction a bien marché sur la base de données, on retrouve le score_directeur :

In [240]:
df.head(5)

,tconst,title,year,runtime,genres,rate,votes,director,casting,index,rank,score_film,score_realisateur
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",0,33273.0,0.001473,0.001473
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",1,6429.0,0.009708,0.012974
2,tt4209788,Le grand jeu,2017,140,"Biography, Crime, Drama",7.4,136590,Aaron Sorkin,"['Jessica Chastain', 'Idris Elba', 'Kevin Cost...",276,16762.0,0.003267,0.012974
3,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",2,98476.0,0.000366,0.004158
4,tt0462335,High-Rise,2015,119,"Drama, Sci-Fi",5.6,37449,Ben Wheatley,"['Tom Hiddleston', 'Jeremy Irons', 'Sienna Mil...",2204,121716.0,0.000258,0.004158
